### This notebook will be mainly used for the Coursera capstone project

In [14]:
# Import dependencies

import pandas as pd
import numpy as np

print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Web-Scraping Wikipedia

In [70]:
from bs4 import BeautifulSoup
import requests
from pandas import DataFrame

html = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
contents = requests.get(html)

soup = BeautifulSoup(contents.content, 'lxml')
table = soup.find_all('table')[0]
lists = pd.read_html(str(table))

df = lists[0]

#### Initial Dataframe

In [72]:
print(f'Initial dataset size: {df.shape}')
df.head()

Initial dataset size: (180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Ignoring cells with a borough that is Not assigned.

In [73]:
indexBorough = df[df['Borough'] == 'Not assigned'].index
df.drop(indexBorough, inplace=True)
print(f'Updated dataset size: {df.shape}')
df.head()

Updated dataset size: (103, 3)


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [75]:
temp = df[df['Neighbourhood'] == 'Not assigned']
temp

,Postal Code,Borough,Neighbourhood
